# Homework2

# Task 1

In general:

## Volcano (Tuple-at-a-time) execution:
- has advantages:
  1. No meterialization, no stalling CPU for waiting for fetching data from memory.
  2. Use-case-friendly for disk-based databases, since a tuple on a page in buffer pool is likely to be fully utilized.
- has disadvantages:
  1. Sensitive to complex querys with generating large amount of instructions and number of function calls.
  2. Each operator keeps its own state. All tightly interleaved operators combine large foorprint causing instruction cache misses.
  3. `next()` is a virtual funtion and called for every tuple. Function call overhead is expensive and impossible to be amortized. 
  4. The code is hard for compiler to optimize since the pipelined function calls is relative complex.
  5. SIMD instructions application is hardly possible, since each tuple is handled seperatedly.
  6. Pipeline breaker operators have to block until their children emit all of their tuples.
  
  
## Materialization (Operator-at-a-time) execution has:
- has advantages:
  1. Each operator is executed once. Thus no pipelining and more independece in whole execution.
  2. Not sensitive to query's complexity. The generated instruction size is stable at a relative narrowed size. The instruction as well as opeartor state efficiently fit into cache.
  3. The code pattern is simple for compiler to optimize, e.g. loop unrolling and application of SIMD instructions.
  4. Function call overhead is not a issue. 
- has disadvantages:
  1. Each result as well as intermediate subresult is fully materialized in memory. Such copy stresses the memory bus.
  2. Each operator is potential to generate data having size larger than the cache. Cache-miss-chain-effect possible: if the child provides an input larger than cache to the parent, the output of parent is likely excess the cache size.
  3. For complex querys, mutiple materializations is unavoidable.

The aforementioned advantages and disadvantages are refered and reflected in the following expermeints' metrics.

## Experiments

The experiments are performed on a Server with Intel(R) Xeon(R) CPU E5-2660 v2 @ 2.20GHz and 220 GiB main memory.

The selected queries are executed on a relation with single column of `200000000` `long int` elements. Each `long int` on 64bit machine has 8 bytes size. Thus the relation size is about 1.5 GiB.

### Query 0

```sql
SELECT SUM(x) FROM rel WHERE x <> 11 AND x <> 42 AND x <> 99　AND x <> 30 AND x <> 77;
```


| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 4325.8  |  56.23 | 114.00        | 0.14       | 0.01        | 0.07            | 21.63       | 2.03 | 2.60  |
| Materialization (Operator-at-a-time) | 2183.8  | 32.01  | 36.06         | 1.63       | 0.38        | 0.06            | 10.92        | 1.13 | 2.93 |

### Query 1

```sql
SELECT x FROM rel WHERE x == 11;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 831.8  | 10.80  | 19.95          | 0.15       | 0.01        | 0.01            | 4.15       | 1.85 | 2.60  |
| Materialization (Operator-at-a-time) | 1237.4 | 16.08  | 9.88           | 0.42       | 0.17        | 0.01            | 6.19       | 0.61 | 2.60  |

### Query 2

```sql
SELECT SUM(x) FROM rel WHERE x <> 12 AND x <> 11 AND x <> 42 AND x <> 43;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 3537.4 | 46.74   | 95.94         | 0.14       | 0.01        | 0.05            | 17.69       | 2.05 | 2.64  |
| Materialization (Operator-at-a-time) | 2042.6 | 28.03   | 30.74         | 1.39       | 0.34        | 0.05            | 10.21       | 1.10 | 2.74  |


### Query 3
```sql
SELECT sum(x) FROM rel;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 843.9  | 10.97   | 21.78         | 0.14       | 0.01        | 0.00            | 4.22        | 1.99 | 2.60  |
| Materialization (Operator-at-a-time) | 1153.6 | 14.99   | 8.69          | 1.42       | 0.20        | 0.00            | 5.77        | 0.58 | 2.60  |

We try to capture some general patterns from the 4 selected queries:
- The Volcano execution model has more instructions than the Materialization execution model. Within the 4 queries a similar pattern is oberserd: the Volcano execution model's instructions are 3x time more than the Materialization execution model.\[ Recall on Volcano disadvantage 1, Materialization advantage 2 above \] However, more instructions is a not factor, which determines runtime performance directly. There are other factors as IPC (instructions per cycle) and LLC-misses ratio, which also play a important role on runtime.
- As mentioned, the Materialization execution model has much heaviler stress on memory access, since each operator fronts with the whole output, which can be potentially 1 GiB level size in our experiments. This influence results in higher L1-misses ratio and LLC(last level cache)-misses ratio of the Materialization execution model. The L1-cache usage indicates the locality efficiency, while the LLC-cache indicating the main-memory access with cache-misses in all cache levels. .\[ Recall on Volcano advantage 1, Materialization disadvantage 1 and 2 above \]   
- The Volcano execution model has a higher IPC. The reason is each instruction of the Volcano execution model is relative lightweighted. Refering the test book and our context, the Volcano execution model handels only 8 bytes each on each operator, while the other model suffering from potential about 1GiB level memory access. The L1-cache-misses ratio and LLC-misses ratio have a great impact on runtime performance of the Materialization execution model, which has to stall CPU waiting for cache latency even memory latency and results in less instruction per cycle on average. 
- Both execution models share a relative low branch-misses ratio. The reasion for that is our experiment is relative simple coded without many cases to jump into. This applies to our simplified dataset without any edges cases like variable-size tuples / records. Thus, in our experments branch prediction is not a factor, that we are focusing on. 


Then we zoom into each query regarding its runtime performance:
- The query 0 and query 2 are quite similar considering their query pattern: multiple predicates. Waling down into the implementation, the Volcano execution model generates a long function call stack to evaluate the list of predicates, which increases the number of instuctions and virtual function call overhead. As for the implementation of the Materialization execution model, the number of instructions is impervious to the length of predicate list, while the data copy overhead increaing. The Materialization execution model turns out to be competent for such query pattern.
- The query 1 and query 3 share common query pattern: sinple predicate or non predicate. Then the Volcano execution model has the better runtime performance, since the number of instructions is not a difference compared with the other. This gap is rescued by efficent usage of all cache levels.
- As a intermediate conclusion, the absolute number of instruction is the key to determine the performance. This factor depends on the length of predicate list in our experiments. However, the number of instruction ratio of both cases has no great impact, since this ratio remains relative same (about factor 3) in our experiments. The absolute number of instruction of the Volcano execution model is sensitive to the length of predicate list. Once the absolute number of instruction is large, the cache efficiency is not helpful to rescue the runtime performance. Under the hood, the absolute number of instruction reflects the virtual function call overhead, which directly influences the runtime performance.

# Task 1 -- additional

## Experiments
In this additional part, the selected queries are executed on a relation with single column of `1024` `long int` elements. Each `long int` on 64bit machine has 8 bytes size. Thus the relation size is 8KiB, which is quite different use case as the main experiment.
The data size fits into L1-cache size but not into cache-line size. We can expect the Materialization execution model suffers less L1-cache-miss and LLC-cache-miss, also the IPC difference gap is shortened, since the memory latency penalty is excluded.

### Query 0

```sql
SELECT SUM(x) FROM rel WHERE x <> 11 AND x <> 42 AND x <> 99　AND x <> 30 AND x <> 77;
```


| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 0.2     | 105.53 | 130.90        | 0.36       | 0.05        | 0.34            | 38.42       | 1.24 | 2.75 |
| Materialization (Operator-at-a-time) | 0.2     | 64.56  | 51.41         | 0.53       | 0.16        | 0.33            | 23.15       | 0.80 | 2.79 |

### Query 1

```sql
SELECT x FROM rel WHERE x == 11;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 0.2     | 57.85  | 39.39         | 0.44       | 0.02        | 0.22            | 20.56       | 0.68 | 2.81 |
| Materialization (Operator-at-a-time) | 0.2     | 52.22  | 25.26         | 0.52       | 0.01        | 0.22            | 18.11       | 0.48 | 2.88 |

### Query 2

```sql
SELECT SUM(x) FROM rel WHERE x <> 12 AND x <> 11 AND x <> 42 AND x <> 43;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 0.2     | 91.93  | 113.6         | 0.33       | 0.01        | 0.30            | 33.36       | 1.24 | 2.76 |
| Materialization (Operator-at-a-time) | 0.2     | 62.81  | 47.19         | 0.57       | 0.07        | 0.31            | 22.23       | 0.75 | 2.83 |


### Query 3
```sql
SELECT sum(x) FROM rel;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 0.2     | 53.99  | 39.60         | 0.32       | 0.00        | 0.21            | 19.01       | 0.73 | 2.84 |
| Materialization (Operator-at-a-time) | 0.2     | 51.31  | 24.03         | 0.50       | 0.04        | 0.20            | 17.21       | 0.47 | 2.98 |

The result matches our expectation, that the IPC difference between both cases is milded, since the L1-misses are LLC-misses of both cases differ only in a shortened range. In this additional experiments, the memory latency is not a factor that needs to be focused on.

# Task 3

## Experiments

The experiments are performed on a Server with Intel(R) Xeon(R) CPU E5-2660 v2 @ 2.20GHz and 220 GiB main memory.

The selected queries are executed on a relation with single column of `200000000` `long int` elements. Each `long int` on 64bit machine has 8 bytes size. Thus the relation size is about 1.5 GiB.

### Query 0

```sql
SELECT SUM(x) FROM rel WHERE x <> 11 AND x <> 42 AND x <> 99　AND x <> 30 AND x <> 77;
```

| Batch Size (Vector Size)             | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock   | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| 256                                  | 3199.6  | 43.10  | 119.89        | 0.14       | 0.03        | 0.11            | 16.00        | 2.78 | 2.69 |
| 512                                  | 3172.2  | 41.96  | 119.22        | 0.22       | 0.02        | 0.08            | 15.96        | 2.84 | 2.65 |
| 1024                                 | 3025.3  | 42.00  | 118.88        | 0.95       | 0.02        | 0.07            | 15.12        | 2.83 | 2.78 |
| 2048                                 | 3205.5  | 41.80  | 118.71        | 1.32       | 0.04        | 0.06            | 16.03        | 2.84 | 2.61 |
| 4096                                 | 3107.8  | 41.96  | 118.63        | 1.62       | 0.04        | 0.06            | 15.54        | 2.83 | 2.70 |
| 8192                                 | 3243.1  | 42.29  | 118.59        | 1.62       | 0.05        | 0.05            | 16.21        | 2.80 | 2.61 |
| 16384                                | 3247.0  | 42.46  | 118.57        | 1.61       | 0.05        | 0.05            | 16.23        | 2.79 | 2.62 |
| 32768                                | 3313.3  | 43.52  | 118.56        | 1.61       | 0.04        | 0.05            | 16.56        | 2.72 | 2.63 |
| 65536                                | 3342.1  | 43.44  | 118.55        | 1.61       | 0.04        | 0.05            | 16.71        | 2.73 | 2.60 |
| 131072                               | 3334.2  | 43.34  | 118.57        | 1.61       | 0.04        | 0.05            | 16.67        | 2.74 | 2.60 |
| 262144                               | 3323.9  | 43.22  | 118.58        | 1.61       | 0.04        | 0.05            | 16.62        | 2.74 | 2.60 |
| 524288                               | 3361.1  | 43.69  | 118.61        | 1.61       | 0.06        | 0.05            | 16.80        | 2.72 | 2.60 |
| 1048576                              | 3415.7  | 44.40  | 118.66        | 1.61       | 0.11        | 0.05            | 17.08        | 2.67 | 2.60 |
| 2097152                              | 3461.5  | 45.00  | 118.79        | 1.61       | 0.14        | 0.05            | 17.31        | 2.64 | 2.60 |
| 4194304                              | 3492.8  | 45.71  | 118.97        | 1.61       | 0.15        | 0.05            | 17.46        | 2.61 | 2.61 |
| 8388608                              | 3589.5  | 46.77  | 119.47        | 1.61       | 0.17        | 0.05            | 17.95        | 2.55 | 2.61 |
| 16777216                             | 3770.6  | 49.21  | 120.45        | 1.62       | 0.20        | 0.05            | 18.85        | 2.45 | 2.61 |
| 33554432                             | 4150.3  | 53.95  | 122.42        | 1.64       | 0.24        | 0.05            | 20.75        | 2.27 | 2.60 |


### Query 1

```sql
SELECT x FROM rel WHERE x == 11;
```

| Batch Size (Vector Size)             | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock   | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| 256                                  | 624.8   | 8.58   | 16.89         | 0.14       | 0.04        | 0.02            | 3.12         | 1.97 | 2.75 |
| 512                                  | 623.8   | 8.68   | 16.78         | 0.14       | 0.04        | 0.01            | 3.11         | 1.93 | 2.79 |
| 1024                                 | 563.9   | 8.10   | 16.73         | 0.15       | 0.03        | 0.01            | 2.81         | 2.06 | 2.88 |
| 2048                                 | 571.2   | 7.86   | 16.70         | 0.18       | 0.03        | 0.01            | 2.85         | 2.13 | 2.76 |
| 4096                                 | 560.0   | 7.81   | 16.68         | 0.39       | 0.03        | 0.01            | 2.79         | 2.14 | 2.79 |
| 8192                                 | 575.5   | 8.32   | 16.68         | 0.40       | 0.04        | 0.01            | 2.87         | 2.01 | 2.90 |
| 16384                                | 590.3   | 8.41   | 16.67         | 0.40       | 0.04        | 0.01            | 2.84         | 1.98 | 2.86 |
| 32768                                | 641.6   | 8.73   | 16.67         | 0.40       | 0.04        | 0.01            | 3.20         | 1.91 | 2.73 |
| 65536                                | 614.1   | 8.30   | 16.66         | 0.40       | 0.04        | 0.01            | 3.07         | 2.01 | 2.71 |
| 131072                               | 649.0   | 8.74   | 16.67         | 0.40       | 0.04        | 0.01            | 3.24         | 1.91 | 2.70 |
| 262144                               | 633.6   | 8.34   | 16.68         | 0.40       | 0.04        | 0.01            | 3.16         | 2.00 | 2.64 |
| 524288                               | 663.0   | 8.68   | 16.69         | 0.40       | 0.04        | 0.01            | 3.31         | 1.92 | 2.62 |
| 1048576                              | 645.4   | 8.38   | 16.70         | 0.40       | 0.04        | 0.01            | 3.32         | 1.99 | 2.60 |
| 2097152                              | 711.9   | 9.24   | 16.67         | 0.40       | 0.05        | 0.01            | 3.55         | 1.80 | 2.60 |
| 4194304                              | 735.5   | 9.78   | 16.72         | 0.40       | 0.08        | 0.01            | 3.76         | 1.71 | 2.60 |
| 8388608                              | 784.2   | 10.18  | 16.80         | 0.40       | 0.08        | 0.01            | 3.92         | 1.65 | 2.60 |
| 16777216                             | 807.9   | 10.49  | 16.80         | 0.40       | 0.09        | 0.01            | 4.03         | 1.62 | 2.60 |
| 33554432                             | 826.8   | 10.49  | 17.29         | 0.40       | 0.11        | 0.01            | 4.13         | 1.57 | 2.66 |

### Query 2

```sql
SELECT SUM(x) FROM rel WHERE x <> 12 AND x <> 11 AND x <> 42 AND x <> 43;
```

| Batch Size (Vector Size)             | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock   | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| 256                                  | 2768.0  | 36.38  | 99.67         | 0.14       | 0.03        | 0.10            | 13.84        | 2.74 | 2.63 |
| 512                                  | 2658.7  | 35.67  | 99.12         | 0.17       | 0.02        | 0.07            | 13.29        | 2.78 | 2.68 |
| 1024                                 | 2466.6  | 36.00  | 98.84         | 0.73       | 0.04        | 0.06            | 12.33        | 2.75 | 2.92 |
| 2048                                 | 2678.7  | 35.67  | 98.70         | 1.10       | 0.04        | 0.05            | 13.39        | 2.77 | 2.66 |
| 4096                                 | 2715.3  | 35.71  | 98.63         | 1.38       | 0.04        | 0.04            | 13.58        | 2.76 | 2.63 |
| 8192                                 | 2634.6  | 36.44  | 98.60         | 1.38       | 0.05        | 0.04            | 13.17        | 2.71 | 2.77 |
| 16384                                | 2606.0  | 36.15  | 98.58         | 1.38       | 0.05        | 0.04            | 13.03        | 2.73 | 2.77 |
| 32768                                | 2764.9  | 37.55  | 98.58         | 1.37       | 0.04        | 0.04            | 13.83        | 2.62 | 2.72 |
| 65536                                | 2739.5  | 36.78  | 98.57         | 1.37       | 0.04        | 0.04            | 13.70        | 2.68 | 2.69 |
| 131072                               | 2780.0  | 37.07  | 98.58         | 1.37       | 0.04        | 0.04            | 13.90        | 2.66 | 2.67 |
| 262144                               | 2715.0  | 37.06  | 98.59         | 1.37       | 0.04        | 0.04            | 13.57        | 2.67 | 2.72 |
| 524288                               | 2785.0  | 37.14  | 98.61         | 1.37       | 0.05        | 0.04            | 13.92        | 2.65 | 2.67 |
| 1048576                              | 2785.0  | 37.14  | 98.61         | 1.37       | 0.11        | 0.04            | 14.32        | 2.58 | 2.67 |
| 2097152                              | 2922.0  | 38.52  | 98.70         | 1.37       | 0.12        | 0.04            | 14.61        | 2.56 | 2.64 |
| 4194304                              | 2980.6  | 39.07  | 98.90         | 1.37       | 0.14        | 0.04            | 14.90        | 2.53 | 2.62 |
| 8388608                              | 3067.0  | 40.18  | 99.32         | 1.38       | 0.16        | 0.04            | 15.33        | 2.47 | 2.62 |
| 16777216                             | 3195.5  | 41.90  | 100.13        | 1.38       | 0.18        | 0.04            | 15.98        | 2.39 | 2.62 |
| 33554432                             | 3561.0  | 46.43  | 101.78        | 1.39       | 0.21        | 0.04            | 17.80        | 2.19 | 2.61 |


### Query 3
```sql
SELECT sum(x) FROM rel;
```

| Batch Size (Vector Size)             | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock   | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| 256                                  | 770.4   | 11.05  | 16.76         | 0.14       | 0.04        | 0.00            | 3.85         | 1.52 | 2.87 |
| 512                                  | 792.0   | 11.30  | 16.65         | 0.14       | 0.04        | 0.00            | 3.96         | 1.46 | 2.86 |
| 1024                                 | 740.1   | 10.58  | 16.59         | 0.14       | 0.03        | 0.00            | 3.70         | 1.57 | 2.86 |
| 2048                                 | 786.7   | 11.77  | 16.56         | 0.17       | 0.03        | 0.00            | 3.93         | 1.41 | 2.99 |
| 4096                                 | 768.2   | 11.11  | 16.55         | 0.38       | 0.04        | 0.00            | 3.84         | 1.49 | 2.89 |
| 8192                                 | 756.6   | 10.96  | 16.54         | 0.40       | 0.03        | 0.00            | 3.78         | 1.51 | 2.90 |
| 16384                                | 775.9   | 11.38  | 16.53         | 0.40       | 0.04        | 0.00            | 3.88         | 1.45 | 2.93 |
| 32768                                | 793.4   | 11.48  | 16.54         | 0.40       | 0.04        | 0.00            | 3.97         | 1.44 | 2.90 |
| 65536                                | 811.2   | 11.74  | 16.53         | 0.40       | 0.04        | 0.00            | 4.06         | 1.41 | 2.90 |
| 131072                               | 846.5   | 12.27  | 16.52         | 0.40       | 0.04        | 0.00            | 4.23         | 1.35 | 2.90 |
| 262144                               | 817.5   | 11.11  | 16.54         | 0.40       | 0.03        | 0.00            | 4.09         | 1.49 | 2.72 |
| 524288                               | 781.9   | 11.24  | 16.54         | 0.40       | 0.03        | 0.00            | 3.91         | 1.47 | 2.88 |
| 1048576                              | 823.8   | 11.50  | 16.55         | 0.40       | 0.04        | 0.00            | 4.12         | 1.44 | 2.79 |
| 2097152                              | 819.2   | 11.85  | 16.50         | 0.40       | 0.05        | 0.00            | 4.09         | 1.39 | 2.89 |
| 4194304                              | 873.2   | 12.70  | 16.54         | 0.40       | 0.06        | 0.00            | 4.36         | 1.30 | 2.91 |
| 8388608                              | 871.3   | 12.60  | 16.62         | 0.40       | 0.07        | 0.00            | 4.35         | 1.32 | 2.89 |
| 16777216                             | 935.8   | 13.22  | 16.79         | 0.40       | 0.09        | 0.00            | 4.68         | 1.27 | 2.83 |
| 33554432                             | 950.1   | 13.66  | 17.12         | 0.40       | 0.09        | 0.00            | 4.75         | 1.25 | 2.88 |

The experiments above cover plenty of cases of batch size settings on 4 selected queries. Since the batch size (vector size) is the only one factor that varys, metrics as number of instructions and branch-misses remain stable in all cases.

A common pattern is infered from the selected 4 queries with varying batch size (vector size): the L1-misses and LLC-misses ratio increase, when batch size (vector size) increases. That is clear, when the batch size grows beyond the cache line size, L1 cache size even LLC size, the more misses on corresponding cache level can be found. Thus, the runtime performance suffers from the latency from cache and memory. Among all the cases, the case with batch size 1024 is relative performant and has cache efficientcy property. We select this case and compare with the aforementioned two execution models.

## Volcano, Materialization, Vectorization

### Query 0

```sql
SELECT SUM(x) FROM rel WHERE x <> 11 AND x <> 42 AND x <> 99　AND x <> 30 AND x <> 77;
```


| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 4325.8  |  56.23 | 114.00        | 0.14       | 0.01        | 0.07            | 21.63       | 2.03 | 2.60  |
| Materialization (Operator-at-a-time) | 2183.8  | 32.01  | 36.06         | 1.63       | 0.38        | 0.06            | 10.92        | 1.13 | 2.93 |
| Vectorization (Vector-at-a-time) with batch size 1024 | 3025.3  | 42.00  | 118.88        | 0.95       | 0.02        | 0.07            | 15.12        | 2.83 | 2.78 |

### Query 1

```sql
SELECT x FROM rel WHERE x == 11;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 831.8  | 10.80  | 19.95          | 0.15       | 0.01        | 0.01            | 4.15       | 1.85 | 2.60  |
| Materialization (Operator-at-a-time) | 1237.4 | 16.08  | 9.88           | 0.42       | 0.17        | 0.01            | 6.19       | 0.61 | 2.60  |
| Vectorization (Vector-at-a-time) with batch size 1024 | 563.9   | 8.10   | 16.73         | 0.15       | 0.03        | 0.01            | 2.81         | 2.06 | 2.88 |

### Query 2

```sql
SELECT SUM(x) FROM rel WHERE x <> 12 AND x <> 11 AND x <> 42 AND x <> 43;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 3537.4 | 46.74   | 95.94         | 0.14       | 0.01        | 0.05            | 17.69       | 2.05 | 2.64  |
| Materialization (Operator-at-a-time) | 2042.6 | 28.03   | 30.74         | 1.39       | 0.34        | 0.05            | 10.21       | 1.10 | 2.74  |
| Vectorization (Vector-at-a-time) with batch size 1024 | 2466.6  | 36.00  | 98.84         | 0.73       | 0.04        | 0.06            | 12.33        | 2.75 | 2.92 |


### Query 3
```sql
SELECT sum(x) FROM rel;
```

| Type                                 | Runtime | cycles | instructions  |  L1-misses | LLC-misses  | branch-misses   | task-clock  | IPC  |  GHz |
|---|---|---|---|---|---|---|---|---|---|
| Volcano (Tuple-at-a-time)            | 843.9  | 10.97   | 21.78         | 0.14       | 0.01        | 0.00            | 4.22        | 1.99 | 2.60  |
| Materialization (Operator-at-a-time) | 1153.6 | 14.99   | 8.69          | 1.42       | 0.20        | 0.00            | 5.77        | 0.58 | 2.60  |
| Vectorization (Vector-at-a-time) with batch size 1024 | 740.1   | 10.58  | 16.59         | 0.14       | 0.03        | 0.00            | 3.70         | 1.57 | 2.86 |



As learned in the lecture and textbook, the Vectorization execution model combines the concept from the Volcano and Materialization execution models. 
- The number of instructions of the Vectorization execution model stays at same level as the Volcano model, but achieves a better performance in all cases than the Volcano model. The reason for that, the virtual function call overhead is hidden and amortized by the Vectorization execution model. In this context, a single virtual function call is batched for 1024 tuples. 
- The Vectorization execution model inherits the cache efficiency property of the Volcano execution model. The LLC-misses ratio remain at the same level as the Volcano execution model, while the L1-misses ratio slightly higher only for queries with multiple predicates. Overall, the cache efficiency of the Vectorization execution model is more than an order of magnitude better than the Materialization execution model, while the later suffering memory latency and GiB level data copying at each operator.